In [15]:
#https://developers.google.com/earth-engine/tutorials/community/intro-to-python-api

In [29]:
import ee
import pprint
# import geemap
# import folium

In [2]:
ee.Authenticate()
ee.Initialize()
print(ee.Image("NASA/NASADEM_HGT/001").get("title").getInfo())

Enter verification code: 4/1AfgeXvspsJPcWpRc3xCrIsSz3W-knArQQIb5jOU3_XHMIgq2AmgYAfI02_A

Successfully saved authorization token.
NASADEM: NASA NASADEM Digital Elevation 30m


In [13]:
# Import the MODIS land cover collection.
lc = ee.ImageCollection('MODIS/006/MCD12Q1')
# Import the MODIS land surface temperature collection.
lst = ee.ImageCollection('MODIS/006/MOD11A1')
# Import the USGS ground elevation image.
elv = ee.Image('USGS/SRTMGL1_003')

In [52]:
# Initial date of interest (inclusive).
i_date = '2017-01-01'

# Final date of interest (exclusive).
f_date = '2020-01-01'

# Selection of appropriate bands and dates for LST.
lst = lst.select('LST_Day_1km').filterDate(i_date, f_date)
lst = lst.select('LST_Day_1km').filterDate(i_date)

In [17]:
# Define the urban location of interest as a point near Lyon, France.
u_lon = 4.8148
u_lat = 45.7758
u_poi = ee.Geometry.Point(u_lon, u_lat)

# Define the rural location of interest as a point away from the city.
r_lon = 5.175964
r_lat = 45.574064
r_poi = ee.Geometry.Point(r_lon, r_lat)

In [20]:
scale = 10000  # scale in meters

# Print the elevation near Lyon, France.
elv_urban_point = elv.sample(u_poi, scale).first().get('elevation').getInfo()
print('Ground elevation at urban point:', elv_urban_point, 'm')

# Calculate and print the mean value of the LST collection at the point.
lst_urban_point = lst.mean().sample(u_poi, scale).first().get('LST_Day_1km').getInfo()
print('Average daytime LST at urban point:', round(lst_urban_point*0.02 -273.15, 2), '°C')

# Print the land cover type at the point.
lc_urban_point = lc.first().sample(u_poi, scale).first().get('LC_Type1').getInfo()
print('Land cover value at urban point is:', lc_urban_point)

Ground elevation at urban point: 178 m
Average daytime LST at urban point: 21.67 °C
Land cover value at urban point is: 9


In [43]:
# Calculate and print the mean value of the LST collection at the point.
lst_urban_point = lst.mean().sample(u_poi, scale).first().get('LST_Day_1km').getInfo()
print('Average daytime LST at urban point:', round(lst_urban_point*0.02 -273.15, 2), '°C')

Average daytime LST at urban point: 21.67 °C


In [53]:
# Get the data for the pixel intersecting the point in urban area.
lst_u_poi = lst.getRegion(u_poi, scale).getInfo()

# Get the data for the pixel intersecting the point in rural area.
lst_r_poi = lst.getRegion(r_poi, scale).getInfo()

# Preview the result.
lst_u_poi[:5]

[['id', 'longitude', 'latitude', 'time', 'LST_Day_1km'],
 ['2017_01_01', 4.80598677003944, 45.76916372588962, 1483228800000, None]]

In [98]:
#Cynthia attempt 
date = '2019-01-01'

#dataset: 
era = ee.ImageCollection("ECMWF/ERA5/MONTHLY") 

print(era.bands())

AttributeError: 'ImageCollection' object has no attribute 'bands'

In [88]:
def ee_array_to_df(arr, list_of_bands):
    """Transforms client-side ee.Image.getRegion array to pandas.DataFrame."""
    df = pd.DataFrame(arr)

    # Rearrange the header.
    headers = df.iloc[0]
    df = pd.DataFrame(df.values[1:], columns=headers)

    # Remove rows without data inside.
    df = df[['longitude', 'latitude', 'time', *list_of_bands]].dropna()

    # Convert the data to numeric values.
    for band in list_of_bands:
        df[band] = pd.to_numeric(df[band], errors='coerce')

    # Convert the time field into a datetime.
    df['datetime'] = pd.to_datetime(df['time'], unit='ms')

    # Keep the columns of interest.
    df = df[['time','datetime',  *list_of_bands]]

    return df

In [117]:
#get info for one location, at one day time 
import pandas as pd


# day = era.select('mean_2m_air_temperature').filterDate(date)
# day = era.select('mean_2m_air_temperature').filterDate(s_date, e_date) 

day = era.filterDate(date)
# day = era 
u_lon = 1
u_lat = 1
u_poi = ee.Geometry.Point(u_lon, u_lat)
day_info = day.getRegion(u_poi, scale).getInfo()

print(list(day_info[0]))
features = ['mean_2m_air_temperature', 'minimum_2m_air_temperature', 'maximum_2m_air_temperature', 'dewpoint_2m_temperature', 'total_precipitation', 'surface_pressure', 'mean_sea_level_pressure', 'u_component_of_wind_10m', 'v_component_of_wind_10m']
ee_array_to_df(day_info, features)

['id', 'longitude', 'latitude', 'time', 'mean_2m_air_temperature', 'minimum_2m_air_temperature', 'maximum_2m_air_temperature', 'dewpoint_2m_temperature', 'total_precipitation', 'surface_pressure', 'mean_sea_level_pressure', 'u_component_of_wind_10m', 'v_component_of_wind_10m']


,time,datetime,mean_2m_air_temperature,minimum_2m_air_temperature,maximum_2m_air_temperature,dewpoint_2m_temperature,total_precipitation,surface_pressure,mean_sea_level_pressure,u_component_of_wind_10m,v_component_of_wind_10m
0,1546300800000,2019-01-01,300.241974,298.426941,301.570068,296.943604,0.127314,101166.109375,101166.195312,1.736544,3.23415
